In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shobika\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [7]:
import numpy as np 
import pandas as pd
df = pd.read_csv('status text-2018.csv', delimiter = '\t')
df.head

<bound method NDFrame.head of                                               status_text
0       Lead Vehicle - Model Year changed from "Nano 2...
1                  EK awaiting lab results (micrographs).
2           Lab results returned and reviewing on 29-Jan.
3       Results from lab show that no contamination or...
4       EK running additional FEA modeling.  Looking a...
...                                                   ...
310099  PAT changed from "Accommodation & Usage" to "D...
310100  System changed from "1 Body Structures" to "PM...
310101  PDL is updated, and no issued so far.??request...
310102  AIM pend and close as per PDL was updated. The...
310103  AIM move to close as per issue validated by K....

[310104 rows x 1 columns]>

In [16]:
def preprocess_text(text: str, remove_stopwords: bool) -> str:
# remove links
    text = re.sub(r"http\S+", "", text)
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text)
    # remove stopwords
    if remove_stopwords:
        # 1. tokenize
        tokens = nltk.word_tokenize(text)
        # 2. check if stopword
        tokens = [w for w in tokens if not w.lower() in stopwords.words("english")]
        # 3. join back together
        text = " ".join(tokens)
    # return text in lower case and stripped of whitespaces
    text = text.lower().strip()
    return text

In [18]:
df['cleaned'] = df['corpus'].apply(lambda x: preprocess_text(x, remove_stopwords=True))

KeyError: 'corpus'

In [8]:
corpus = []
stop_words = set(stopwords.words("english"))
for i in range(0,1000):
    status_text = re.sub('[^a-zA-Z]', ' ', df['status_text'][i])
    status_text = status_text.lower()
    status_text = status_text.split()
    ps = PorterStemmer()
    status_text = [ps.stem(word) for word in status_text
                if not word in set(stopwords.words('english'))]
    status_text = ' '.join(status_text) 
    corpus.append(status_text)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans

In [10]:
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=5, max_df=0.95)
# fit_transform applies TF-IDF to clean texts - we save the array of vectors in X
X = vectorizer.fit_transform(df['status_text'])

In [11]:
from sklearn.cluster import KMeans

# initialize kmeans with 3 centroids
kmeans = KMeans(n_clusters=3, random_state=42)
# fit the model
kmeans.fit(X)
# store cluster labels in a variable
clusters = kmeans.labels_

In [13]:
from sklearn.decomposition import PCA

# initialize PCA with 2 components
pca = PCA(n_components=2, random_state=42)
# pass our X to the pca and store the reduced vectors into pca_vecs
pca_vecs = pca.fit_transform(X.toarray())
# save our two dimensions into x0 and x1
x0 = pca_vecs[:, 0]
x1 = pca_vecs[:, 1]

MemoryError: Unable to allocate 43.0 GiB for an array with shape (310104, 18600) and data type float64

In [12]:
def get_top_keywords(n_terms):
    """This function returns the keywords for each centroid of the KMeans"""
    df = pd.DataFrame(X.todense()).groupby(clusters).mean() # groups the TF-IDF vector by cluster
    terms = vectorizer.get_feature_names_out() # access tf-idf terms
    for i,r in df.iterrows():
        print('\nCluster {}'.format(i))
        print(','.join([terms[t] for t in np.argsort(r)[-n_terms:]])) # for each row of the dataframe, find the n terms that have the highest tf idf score
            
get_top_keywords(10)

MemoryError: Unable to allocate 43.0 GiB for an array with shape (310104, 18600) and data type float64

In [15]:
print(stopwords)

<WordListCorpusReader in 'C:\\Users\\Shobika\\AppData\\Roaming\\nltk_data\\corpora\\stopwords'>


In [ ]:
# from sklearn.cluster import KMeans
# Sum_of_squared_distances = []
# K = range(2,10)
# for k in K:
#     km = KMeans(n_clusters=k, max_iter=200, n_init=10)
#     km = km.fit(X)
#     Sum_of_squared_distances.append(km.inertia_)
# plt.plot(K, Sum_of_squared_distances, 'bx-')
# plt.xlabel('k')
# plt.ylabel('Sum_of_squared_distances')
# plt.title('Elbow Method For Optimal k')
# plt.show()

# print('How many clusters do you want to use?')
# true_k = int(input())
# model = KMeans(n_clusters=true_k, init='k-means++', max_iter=200, n_init=10)
# model.fit(X)

# labels=model.labels_
# clusters=pd.DataFrame(list(zip(status_text,labels)),columns=['status_text','cluster'])
# #print(clusters.sort_values(by=['cluster']))

# for i in range(true_k):
#     print(clusters[clusters['cluster'] == i])
        
# return

In [ ]:
# from sklearn.decomposition import PCA

# # initialize PCA with 2 components
# pca = PCA(n_components=2, random_state=42)
# # pass our X to the pca and store the reduced vectors into pca_vecs
# pca_vecs = pca.fit_transform(X.toarray())
# # save our two dimensions into x0 and x1
# x0 = pca_vecs[:, 0]
# x1 = pca_vecs[:, 1]
# print(x0,x1)

In [ ]:
# df['cluster'] = clusters
# df['x0'] = x0
# df['x1'] = x1

In [ ]:
def get_top_keywords(n_terms):
    """This function returns the keywords for each centroid of the KMeans"""
    df = pd.DataFrame(X.todense()).groupby(clusters).mean() # groups the TF-IDF vector by cluster
    terms = vectorizer.get_feature_names_out() # access tf-idf terms
    for i,r in df.iterrows():
        print('\nCluster {}'.format(i))
        print(','.join([terms[t] for t in np.argsort(r)[-n_terms:]])) # for each row of the dataframe, find the n terms that have the highest tf idf score
            
get_top_keywords(10)

In [ ]:
plt.figure(figsize=(12, 7))
# set a title
plt.title("TF-IDF + KMeans 20newsgroup clustering", fontdict={"fontsize": 18})
# set axes names
plt.xlabel("X0", fontdict={"fontsize": 16})
plt.ylabel("X1", fontdict={"fontsize": 16})
# create scatter plot with seaborn, where hue is the class used to group the data
sns.scatterplot(data=df, x='x0', y='x1', hue='cluster', palette="viridis")
plt.show()